# QASSIS: Quality Algorithm to Search Stuff in Spectra

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sig
from lmfit import minimize, Parameters, fit_report, Model, Parameter
import corner as corner
#%matplotlib inline
#from pylab import rcParams
#rcParams['figure.figsize'] = 15, 10
%matplotlib qt5

In [6]:
# Get spectrum file
spectrum_file = "181_WSW_FTS200_3mm_average_data_Tmb.dat"
spectrum = np.loadtxt(spectrum_file)

frequencies = spectrum[:,0]
intensities = spectrum[:,1]

# Get molecular catalog file
molecule_name = "HC3N"
molecule_file = "HC3N.txt"
molecule_catalog = pd.read_csv(molecule_file, delimiter="\t", header=0, index_col=False)

molecule_frequencies = molecule_catalog["Frequency(Ghz)"]
molecule_intensities = molecule_catalog["Intensity(K)"]
molecule_eup = molecule_catalog["Eup(K)"]
molecule_aij = molecule_catalog["Aij"]
molecule_transition = molecule_catalog["Transition"]

### OPTIONAL: Redshift spectrum by $v_{lsr}$ and convert to velocity units

In [7]:
reference_frequency = 93750.7299
vlsr = 0#7.1e+3 # m/s
c = 2.998*1e8 # m/s 

def redshift(v):
    return np.sqrt( (1 + (v/c)) / (1 - (v/c)) ) - 1 # = z

#frequencies_shifted = np.array(frequencies)/(1+redshift(-6.8e+3))
frequencies_shifted = frequencies#np.array(frequencies_shifted)/(1+redshift(vlsr))

def freq_to_vel(freqs):
    return np.array((c*reference_frequency - c*freqs) / reference_frequency)
   
velocities = freq_to_vel(frequencies_shifted)/1000 # km/s
plt.step(velocities[intensities>-100], intensities[intensities>-100], color="black", linewidth=0.5)

plt.xlabel("Velocity [km/s]", fontsize=15)
plt.ylabel("Intensity [K]", fontsize=15)


frequencies = freq_to_vel(frequencies_shifted)/1000
molecule_frequencies = freq_to_vel(molecule_catalog["Frequency(Ghz)"])/1000

plt.vlines(molecule_frequencies, 0,1, color="red")
plt.show()


In [8]:
# Find possible emission lines
width = 1

peaks = sig.find_peaks(intensities)[0]
peak_intensities = intensities[peaks]
peak_frequencies = frequencies[peaks]

noise = 2*5e-3
rms = [5*5e-3 for p in peak_frequencies]

emission_line_frequencies = []
emission_line_intensities = []
emission_line_eup = []
emission_line_aij = []
emission_line_transition = []

for i,f1 in enumerate(peak_frequencies):
    for j,f2 in enumerate(molecule_frequencies):
        if abs(f1-f2) < width and peak_intensities[i]>rms[i]:
            emission_line_frequencies.append(f1)
            emission_line_intensities.append(peak_intensities[i])
            emission_line_eup.append(molecule_eup[j])
            emission_line_aij.append(molecule_aij[j])
            emission_line_transition.append(molecule_transition[j])
            
print(emission_line_frequencies)

[67048.6856544463, 8863.58306827433, -20228.655156102413, -49320.26916176575]


# Fit Gaussians to Emission Lines


In [27]:
# Define a window for each line

window_size = 15 # default 5

window_frequencies = []
window_intensities = []

for f in emission_line_frequencies:
    window_f = frequencies[abs(frequencies-f)<window_size]
    window_i = intensities[abs(frequencies-f)<window_size]
    window_frequencies.append(window_f)
    window_intensities.append(window_i)
    
# For each window, define what the BASELINE is

baseline_frequencies = []
baseline_intensities = []

for i,window in enumerate(window_frequencies):
    baseline_window_frequencies = []
    baseline_window_intensities = []
    for j,freq in enumerate(window):
        if window_intensities[i][j] <= abs(noise) and abs(freq-emission_line_frequencies[i]) > 2*width:
            baseline_window_frequencies.append(freq)
            baseline_window_intensities.append(window_intensities[i][j])
    baseline_frequencies.append(baseline_window_frequencies)
    baseline_intensities.append(baseline_window_intensities)
    
# Fit a polynomial to the baseline
degree = 2

baseline_polynomials = []

for i,baseline_window_frequencies in enumerate(baseline_frequencies):
    baseline_fit = np.polyfit(baseline_window_frequencies, baseline_intensities[i], degree)
    baseline = np.poly1d(baseline_fit)
    baseline_polynomials.append(baseline)
    
# Subtract baseline polynomial from window intensities

window_intensities_corrected = []

for i,window in enumerate(window_intensities):
    window_intensities_corrected.append(window_intensities[i]-baseline_polynomials[i](window_frequencies[i]))
    
# For each window, define what the LINE is

line_frequencies = []
line_intensities = []

for i,window in enumerate(window_frequencies):
    line_window_frequencies = []
    line_window_intensities = []
    for j,freq in enumerate(window):
        if abs(freq-emission_line_frequencies[i]) <= 4*width:# and window_intensities_corrected[i][j]>=0:
            window_intensities_corrected_nonzero = [x if x>0 else 0 for x in window_intensities_corrected[i]]
            line_window_frequencies.append(freq)
            line_window_intensities.append(window_intensities_corrected_nonzero[j])
    line_frequencies.append(line_window_frequencies)
    line_intensities.append(line_window_intensities)
    
# Fit a Gaussian to the line

gaussian_fits = []

def gaussian(x, amp, cen, wid):
    """1-d gaussian: gaussian(x, amp, cen, wid)"""
    return (amp / (np.sqrt(2*np.pi) * wid)) * np.exp(-(x-cen)**2 / (2*wid**2))

for i,line_f in enumerate(line_frequencies):
    x = line_frequencies[i]
    y = line_intensities[i]
    
    # Do least squares first
    gmodel = Model(gaussian)
    params = gmodel.make_params()
    
    init_amp = (emission_line_intensities[i]-baseline_polynomials[i](emission_line_frequencies[i]))*np.sqrt(2*np.pi)*0.5
    
    params["amp"] = Parameter(name='amp', value=init_amp, min=init_amp*0.1, max=init_amp*2)
    params["cen"] = Parameter(name='cen', value=emission_line_frequencies[i], min=emission_line_frequencies[i]-3, max=emission_line_frequencies[i]+3)
    params["wid"] = Parameter(name='wid', value=0.5, min=0.1, max=1)
    
    result = gmodel.fit(y, x=x, params=params, method="leastsq")
    
    # Now do MCMC
    
    #emcee_kws = dict(steps=5000, burn=500, thin=10)
    emcee_kws = dict(steps=5000, burn=500, thin=10)
    
    amp = result.best_values["amp"]
    cen = result.best_values["cen"]
    wid = result.best_values["wid"]
    
    gmodel = Model(gaussian)
    params = gmodel.make_params()
    params["amp"] = Parameter(name='amp', value=init_amp, min=init_amp*0.1, max=init_amp*1.5)
    params["cen"] = Parameter(name='cen', value=cen, min=cen-3, max=cen+3)
    params["wid"] = Parameter(name='wid', value=0.3, min=0, max=0.7)
    
    result = gmodel.fit(y, x=x, params=params, method="emcee", fit_kws=emcee_kws)

    
    
    gaussian_fits.append(result)
    print(result.fit_report())

# Define a linspace in each window for plotting purposes

window_linspaces = []
for i,window in enumerate(window_frequencies):
    window_linspaces.append(np.linspace(min(window), max(window),10000))

  0%|          | 12/5000 [00:00<00:41, 119.78it/s]

[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # fitting method   = emcee
    # function evals   = 500000
    # data points      = 13
    # variables        = 3
    chi-square         = 0.35025364
    reduced chi-square = 0.03502536
    Akaike info crit   = -40.9826117
    Bayesian info crit = -39.2877636
[[Variables]]
    amp:  1.35339205 +/- 0.91121812 (67.33%) (init = 2.374853)
    cen:  67048.7350 +/- 1.26594505 (0.00%) (init = 67048.81)
    wid:  0.38297383 +/- 0.25009872 (65.30%) (init = 0.3)


  0%|          | 14/5000 [00:00<00:37, 131.36it/s]

[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # fitting method   = emcee
    # function evals   = 500000
    # data points      = 13
    # variables        = 3
    chi-square         = 0.13375055
    reduced chi-square = 0.01337505
    Akaike info crit   = -53.4974662
    Bayesian info crit = -51.8026182
[[Variables]]
    amp:  0.99383819 +/- 0.67922799 (68.34%) (init = 1.565271)
    cen:  8863.51200 +/- 1.60573726 (0.02%) (init = 8863.482)
    wid:  0.40483039 +/- 0.24952673 (61.64%) (init = 0.3)


  1%|          | 32/5000 [00:00<00:31, 158.31it/s]

[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # fitting method   = emcee
    # function evals   = 500000
    # data points      = 13
    # variables        = 3
    chi-square         = 0.19508876
    reduced chi-square = 0.01950888
    Akaike info crit   = -48.5902499
    Bayesian info crit = -46.8954018
[[Variables]]
    amp:  0.81005480 +/- 0.55415830 (68.41%) (init = 1.222057)
    cen: -20228.4817 +/- 1.69432677 (0.01%) (init = -20228.47)
    wid:  0.39801189 +/- 0.24432824 (61.39%) (init = 0.3)


100%|██████████| 5000/5000 [00:31<00:00, 160.91it/s]


[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # fitting method   = emcee
    # function evals   = 500000
    # data points      = 13
    # variables        = 3
    chi-square         = 0.04933996
    reduced chi-square = 0.00493400
    Akaike info crit   = -66.4616142
    Bayesian info crit = -64.7667662
[[Variables]]
    amp:  0.72928306 +/- 0.45608234 (62.54%) (init = 1.035754)
    cen: -49320.0253 +/- 1.99200673 (0.00%) (init = -49320.12)
    wid:  0.36787412 +/- 0.23857063 (64.85%) (init = 0.3)
[[Correlations]] (unreported correlations are < 0.100)
    C(amp, cen) =  0.147


In [ ]:
print(amp_err)

## Check best fits

In [30]:
for i,line in enumerate(window_intensities):

    plt.subplot2grid((2,3),(i%2,i%3))
    plt.step(window_frequencies[i], window_intensities[i], linewidth=2, c="black")
    plt.step(baseline_frequencies[i], baseline_intensities[i], linewidth=4, c="blue", alpha=0.4)
    plt.scatter(emission_line_frequencies[i], emission_line_intensities[i], s=50, c="red")

    F_linspace = np.linspace(min(window_frequencies[i]), max(window_frequencies[i]),100)
    plt.plot(F_linspace, baseline_polynomials[i](F_linspace), color="blue", linestyle="--", linewidth=3, alpha=0.8)


    plt.step(line_frequencies[i], line_intensities[i], linewidth=5, c="lime", alpha=0.5)

    plt.step(window_frequencies[i], window_intensities_corrected[i], c="red", linewidth=1)

    amp = np.median(gaussian_fits[i].flatchain["amp"])
    cen = np.median(gaussian_fits[i].flatchain["cen"])
    wid = np.median(gaussian_fits[i].flatchain["wid"])
    
    #amp = gaussian_fits[i].best_values["amp"]
    #cen = gaussian_fits[i].best_values["cen"]
    #wid = gaussian_fits[i].best_values["wid"]
    
    gauss = gaussian(window_linspaces[i], amp, cen, wid)
    plt.plot(window_linspaces[i], gauss, linestyle='--', color="magenta", label='best fit', linewidth=3)

    plt.fill_between(window_linspaces[i], y1=[0 for x in window_linspaces[i]], y2 =gauss , color="magenta", alpha=0.5)

plt.xlabel("Velocity [km/s]", fontsize=15)
plt.ylabel("Intensity [K]", fontsize=15)

plt.show()

## Correlation plots to double check _emcee_ didn't go insane

In [29]:
for i,line in enumerate(window_intensities):
    amp = np.median(gaussian_fits[i].flatchain["amp"])
    cen = np.median(gaussian_fits[i].flatchain["cen"])
    wid = np.median(gaussian_fits[i].flatchain["wid"])
    
    amp_err = np.std(gaussian_fits[i].flatchain["amp"])
    cen_err = np.std(gaussian_fits[i].flatchain["cen"])
    wid_err = np.std(gaussian_fits[i].flatchain["wid"])
    
    corner.corner(gaussian_fits[i].flatchain, bins=20, color="indigo", smooth=True, smooth1d=True,
             plot_datapoints=True, plot_density=False, fill_contours=True, levels=[0.67, 0.95,0.99],
                 range=[(amp-10*amp_err, amp+10*amp_err),(cen-10*cen_err, cen+10*cen_err),(wid-10*wid_err, wid+10*wid_err)],
                 labels=["Amplitude", "Center", "Gaussian Width"])

    
    
plt.show()
    

### Derive Rotation Diagrams

In [35]:
# EVERYTHING IN CGS UNITS

# Boltzmann Constant
k = 1.3807 * 1e-16 

# Dipole moment
# CH3CN
#mu = 3.9037 * 1e-18
# CH3OH
# mu = np.sqrt(0.899**2 + (-1.44)**2) * 1e-18
# CH2CO
#mu = 1.422 * 1e-18
# CH3CCH
#mu = 0.784 * 1e-18
# HC3N
mu = 3.730 * 1e-18

# Transition strength
# CH3CN
# strength = np.array([79.48062, 84.80078, 101.74150, 105.98313, 123.64362, 127.17758])/(3.9037**2)
# CH3OH
# strength = np.array([3.08327, 1.21365, 1.61697, 1.61649, 0.97862]) / (np.sqrt(0.899**2 + (-1.44)**2))
# CH2CO
#strength = np.array([28.79393, 9.99777, 28.79435]) / (1.422**2)
# CH3CCH
#strength = np.array([1.79654, 1.87136, 1.99592, 2.18328, 2.24559]) / (0.784 **2)
# HC3N
strength = np.array([111.41550, 139.25442, 153.16666, 167.09368]) / (3.730 **2)

# The integral from -inf to inf is the amp parameter. Convert to K.cm/s

flux = [np.median(fit.flatchain["amp"])*100000 for fit in gaussian_fits] # in K . cm/s
flux_err = [np.std(fit.flatchain["amp"])*100000 for fit in gaussian_fits] # in K . cm/s

centers = [reference_frequency*(c-np.median(fit.flatchain["cen"])*1000)/c *1e6 for fit in gaussian_fits]

# Upper-level populations Nu/gu
# factor of 10000 ti
lnnugu = np.log((3*k*np.array(flux))/(8*(np.pi**3)*np.array(centers)*(mu**2)*np.array(strength)))

nugu_err = ((3*k*np.array(flux_err))/(8*(np.pi**3)*np.array(centers)*(mu**2)*np.array(strength)))
lnnugu_err = nugu_err / np.exp(lnnugu)

emission_line_eup = np.array(emission_line_eup)

In [ ]:
print(centers)

## Fit Rotation Diagram

In [36]:
def residual(params, eu, data, eps_data):
    m = -1/params['T']
    b = params['lnNugu_0']
  
    model = m*eu+b

    return (data-model) / eps_data


params = Parameters()
params.add('T', value=10, min=0, max=100)
params.add('lnNugu_0', value=20, min=15, max=30)


out = minimize(residual, params, args=(np.array(emission_line_eup), lnnugu, lnnugu_err), method="leastsq")

T = out.params["T"]
lnNugu_0 = out.params["lnNugu_0"]

params = Parameters()
params.add('T', value=T, min=T-4*T.stderr, max=T+4*T.stderr)
params.add('lnNugu_0', value=lnNugu_0, min=lnNugu_0-4*lnNugu_0.stderr, max=lnNugu_0+4*lnNugu_0.stderr)

out = minimize(residual, params, args=(np.array(emission_line_eup), lnnugu, lnnugu_err), method="emcee", steps=4000, burn=400, thin=10)


print(fit_report(out))

T = np.median(out.flatchain["T"])
lnNugu_0 = np.median(out.flatchain["lnNugu_0"])

Eu_linspace = np.linspace(min(emission_line_eup), max(emission_line_eup), 1000)

def rotdiag_model(Eu, T, lnNugu_0):
    return lnNugu_0 - (1/T)*Eu

plt.scatter(emission_line_eup, lnnugu, c="purple")

plt.errorbar(emission_line_eup, lnnugu, yerr=lnnugu_err, capsize=5, c="purple", fmt="none")

plt.plot(Eu_linspace, rotdiag_model(Eu_linspace, T, lnNugu_0), color="indigo", linestyle="--")
    
plt.ylabel("$\ln(N_u/g_u)$", fontsize=15)
plt.xlabel("$E_u/k$", fontsize=15)
plt.show()
    


100%|██████████| 4000/4000 [00:38<00:00, 103.63it/s]


[[Fit Statistics]]
    # fitting method   = emcee
    # function evals   = 400000
    # data points      = 4
    # variables        = 2
    chi-square         = 0.05322938
    reduced chi-square = 0.02661469
    Akaike info crit   = -13.2777565
    Bayesian info crit = -14.5051678
[[Variables]]
    T:         13.0817589 +/- 2.34525055 (17.93%) (init = 12.74103)
    lnNugu_0:  25.2073527 +/- 0.39725524 (1.58%) (init = 25.23224)
[[Correlations]] (unreported correlations are < 0.100)
    C(T, lnNugu_0) = -0.705


In [37]:
T_err = np.std(out.flatchain["T"])
lnNugu_0_err = np.std(out.flatchain["lnNugu_0"])
    
    
corner.corner(out.flatchain, bins=20, color="darkcyan", smooth=True, smooth1d=True,
             plot_datapoints=True, plot_density=False, fill_contours=True, levels=[0.67, 0.95,0.99],
                 range=[(T-10*T_err, T+10*T_err),(lnNugu_0-10*lnNugu_0_err, lnNugu_0+10*lnNugu_0_err)],
                 labels=["$T_{rot}$", "$\ln(N_u/g_u)$"])

plt.show()

## Derive total column density $N_{tot}$ by approximating the rotational partition function $Q_{rot}$

In [38]:
# CH3CN
#Qrot = [13.8355, 28.4924, 64.0955, 164.3168, 449.0811, 1267.6705, 2628.0493]
#Trot = [2.725, 5.0, 9.375, 18.75, 37.5, 75.0, 120.0]

# CH3OH
#Qrot = [11.889916, 26.719018, 78.173628, 274.987967, 920.963739, 2924.302297, 9750.039754]
#Trot = [2.725, 5.0, 9.375, 18.75, 37.5, 75.0, 150.0]

# CH2CO
#Qrot = [47.577, 157.217,  457.088,  1285.879, 3433.998]
#Trot = [9.375, 18.75, 37.50, 75.00, 150.0]

# CH3CCH
#Qrot = [68.7664, 176.2982, 481.7197, 1362.3596, 4209.3392]
#Trot = [9.375, 18.75, 37.5, 75.0, 150.0]

# HC3N
Qrot = [43.2767, 86.2186, 172.1063, 343.8929, 687.5086]
Trot = [9.375, 18.75, 37.5, 75.0, 150.0]

Qfit = np.polyfit(Trot, Qrot, 3)
Q = np.poly1d(Qfit)

T_linspace = np.linspace(min(Trot), max(Trot))
plt.scatter(Trot, Qrot, c="black")
plt.plot(T_linspace, Q(T_linspace), color="black")

Qerr = np.sqrt((T*T_err)*(3*(Qfit[0]**2)*(T**4)+2*Qfit[1]*T**2+Qfit[1]**2))

plt.errorbar([T], [Q(T)], yerr=Qerr , c="red", fmt="*", capsize=3)

plt.ylabel("$Q_{rot}$", fontsize=15)
plt.xlabel("$T$", fontsize=15)
plt.show()

In [39]:
lnNtot = lnNugu_0 + np.log(Q(T))
lnNtot_err = np.sqrt((lnNugu_0_err)**2 + (Qerr/Q(T))**2)

logNtot = np.log10(np.exp(lnNtot))
logNtot_err = 0.434*(np.exp(lnNtot)*lnNtot_err/np.exp(lnNtot))

rot_err = np.sqrt((lnNugu_0_err)**2 + (np.array(Eu_linspace)*T**(-2)*T_err)**2)

In [40]:
Eu_linspace = np.linspace(min(emission_line_eup)-5, max(emission_line_eup)+5, 1000)

plt.subplot2grid((1,7), (0,0), colspan=5)

plt.errorbar(emission_line_eup, lnnugu, yerr=lnnugu_err, fmt="o", capsize=3, color="black")

plt.ylabel("$\log(N_u/g_u)$", fontsize=20)
plt.xlabel("$E_u/k$", fontsize=20)

#plt.fill_between(x=Eu_linspace, y1=rotdiag_model(Eu_linspace, T, lnNugu_0)-rot_err, y2=rotdiag_model(Eu_linspace, T, lnNugu_0)+rot_err, alpha=0.1, color="gray")

Tchain_burn = out.flatchain["T"][::10]
lnNugu_0chain_burn = out.flatchain["lnNugu_0"][::10]

for i,T in enumerate(Tchain_burn):
    plt.plot(Eu_linspace, rotdiag_model(Eu_linspace, T, lnNugu_0chain_burn[i*10]), color="lime", linestyle="-", linewidth=0.01, alpha=0.5)
    
T_best = np.median(out.flatchain["T"])
lnNugu_0_best = np.median(out.flatchain["lnNugu_0"])


plt.plot(Eu_linspace, rotdiag_model(Eu_linspace, T_best, lnNugu_0_best), color="red", linestyle="--")
        
plt.ylim([14,30])

plt.subplot2grid((1,7), (0,5), colspan=1)
plt.errorbar([0], [logNtot], yerr=logNtot_err , c="black", capsize=3)

plt.fill_between(x=[-1,1],y1=0,y2=logNtot, color="red", alpha=0.6)
plt.title("$\log_{10}(N_{tot})$", fontsize=20)

plt.xticks([])
plt.ylim([0,20])

plt.subplot2grid((1,7), (0,6), colspan=1)
plt.errorbar([0], [T_best], yerr=T_err , c="black", capsize=3)

plt.fill_between(x=[-1,1],y1=0,y2=T_best, color="red", alpha=0.6)
plt.title("$T_{rot}$(K)", fontsize=20)

plt.xticks([])
plt.ylim([0,50])
plt.xticks(fontsize=18)
plt.show()

In [45]:
round(T_best,4)

13.0818

In [46]:
round(T_err,4)

2.062

In [31]:
emission_line_transition

['HC3N, v=0 (8 _ 7) ',
 'HC3N, v=0 (10 _ 9) ',
 'HC3N, v=0 (11 _ 10) ',
 'HC3N, v=0 (12 _ 11) ']

In [32]:
print(emission_line_eup)

[15.719000000000001, 24.015, 28.818, 34.056999999999995]


In [33]:
for x in (-1*np.array(np.array(emission_line_frequencies)*1000*reference_frequency/c - reference_frequency)):
    print(round(x,3))

72783.875
90978.991
100076.451
109173.716


In [ ]:
print(freq_to_vel(np.array(emission_line_frequencies))/1000)

In [ ]:
molecule_frequencies

In [41]:
for x in lnnugu:
    print(round(x,4))

24.0508
23.2958
22.9008
22.6217


In [42]:
for x in lnnugu_err:
    print(round(x,4))

0.6197
0.5977
0.5902
0.5482


In [47]:
round(logNtot,4)

12.7274

In [48]:
round(logNtot_err,4)

0.1526

In [73]:
abundances = np.array([10**11.6265,10**12.5165, 10**12.7274, 10**14.0074])/10**13.7300  
errors = np.sqrt((np.array([10**0.3773, 10**0.4215, 10**0.1526, 10**0.2384]) / np.array([10**11.6265,10**12.5165, 10**12.7274, 10**14.0074]))**2 + (10**0.2795/10**13.7300)**2)
plt.errorbar([1,2,3,4], abundances, yerr=errors, fmt="s", color="black")
plt.xticks([1,2,3,4], labels=["$CH_3CN$", "$CH_2CO$", "$HC_3N$", "$CH_3CCH$"], fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel("N$_{tot}$ / N$_{tot}$[CH$_3$OH]", fontsize=15)
plt.yscale("log")
plt.show()